In [ ]:
from enum import Enum
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from dotenv import load_dotenv
import os
# 1. 감정 Enum 정의
class Sentiment(Enum):
    긍정 = "긍정"
    부정 = "부정"
    보통 = "보통"

# 2. 출력 파서 (EnumOutputParser)
enum_parser = EnumOutputParser(enum=Sentiment)

# (선택) 분석 실패 시 자동 수정하는 파서
fixing_parser = OutputFixingParser.from_llm(
    parser=enum_parser,
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0)
)
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# 3. 프롬프트 템플릿 정의
template = """
당신은 영화 리뷰 감정을 분류하는 시스템입니다.
리뷰를 읽고 감정을 '긍정', '부정', '보통' 중 하나로만 답변하세요.

리뷰: {review}

{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["review"],
    partial_variables={"format_instructions": enum_parser.get_format_instructions()},
)

# 4. LLM + 파서 체인 정의

# LLM 모델 정의
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

def analyze_review(review: str):
    input_text = prompt.format(review=review)
    output = llm.predict(input_text)
    try:
        return enum_parser.parse(output)
    except:
        # EnumOutputParser 실패 시 OutputFixingParser 사용
        return fixing_parser.parse(output)

# 5. 테스트 리뷰
reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
]

for r in reviews:
    result = analyze_review(r)
    print(f"리뷰: {r}\n감정 분석 결과: {result.value}\n")
